# HW 3

In [1]:
import hashlib
import random
import time
import numpy as np

In [2]:
print("hi :)")

hi :)


In [3]:
import hashlib
import random
import time

# load a file and return its lines as a list
def load_memory_file(filename):
    with open(filename, 'r') as f:
        lines = [line.strip() for line in f if line.strip()]
    return lines

# Load files (adjust the file paths as needed)
random_memory = load_memory_file('random_numbers_16.txt')
zeros_memory = load_memory_file('zeros_8.txt')
attack_memory = load_memory_file('attack_8.txt')


In [4]:
def prover(nonce, memory_lines):
    # Initialize with the nonce as a starting value (convert to string)
    prev_hash = hashlib.sha256(str(nonce).encode()).hexdigest()
    for line in memory_lines:
        # Concatenate previous hash and the current line, then update the hash.
        combined = prev_hash + line
        prev_hash = hashlib.sha256(combined.encode()).hexdigest()
    return prev_hash


In [ ]:
used_nonces = set() # I guess this is one way to do it...

def generate_nonce():
    nonce = random.randint(0, 256)
    while nonce in used_nonces:
        nonce = random.randint(0, 256)
    used_nonces.add(nonce)
    return nonce

def verifier(prover_function, memory_lines):
    nonce = generate_nonce()
    # Start timer
    start_time = time.perf_counter()
    # Get hash from prover
    prover_hash = prover_function(nonce, memory_lines)
    # Compute expected hash locally using the same algorithm
    expected_hash = prover(nonce, memory_lines)
    # End timer
    end_time = time.perf_counter()
    attestation_time = end_time - start_time
    # Check if the attestation passes
    passed = (prover_hash == expected_hash)
    return passed, attestation_time, nonce, expected_hash, prover_hash

# Example run:
passed, att_time, nonce, exp_hash, prov_hash = verifier(prover, random_memory)
print(f"Attestation passed: {passed}, Time: {att_time:.6f} sec")


Attestation passed: True, Time: 0.002245 sec


In [6]:
def performance_analysis(prover_function, memory_lines, runs=10):
    times = []
    for i in range(runs):
        _, att_time, _, _, _ = verifier(prover_function, memory_lines)
        times.append(att_time)
    average_time = sum(times) / len(times)
    return times, average_time

times, avg_time = performance_analysis(prover, random_memory)
print("Individual attestation times:", times)
print(f"Average attestation time: {avg_time:.6f} sec")


Individual attestation times: [0.0011598250002862187, 0.0012267310003153398, 0.0011961749996771687, 0.0013689380002688267, 0.0011684199998853728, 0.0012333539998508058, 0.0012662489998547244, 0.0014393800001926138, 0.0012367210001684725, 0.001320067999586172]
Average attestation time: 0.001262 sec


In [7]:
def precompute_attack_data(zeros_lines, attack_lines):
    # Replace zero values in attack_lines with corresponding non-zero values from zeros_lines
    modified_attack = []
    for zero, attack in zip(zeros_lines, attack_lines):
        # Replace if the attack entry is zero (as a string or numeric zero)
        if attack == "0" or attack == 0:
            modified_attack.append(zero)
        else:
            modified_attack.append(attack)
    return modified_attack



In [8]:
def malicious_prover(nonce, attack_memory_lines, zeros_memory_lines):
    # Compute the modified data by replacing zeros in attack data with non-zero values from zeros_memory_lines.
    modified_data = precompute_attack_data(zeros_memory_lines, attack_memory_lines)
    # Compute the cumulative SHA-256 hash (using the same algorithm as the legitimate prover)
    return prover(nonce, modified_data)

# Create a wrapper that only takes two arguments
malicious_wrapper = lambda nonce, memory_lines: malicious_prover(nonce, memory_lines, zeros_memory)

# Measure malicious prover performance over 10 runs using the wrapper.
malicious_times, malicious_avg = performance_analysis(malicious_wrapper, attack_memory)
print("Malicious prover times:", malicious_times)
print(f"Average malicious attestation time: {malicious_avg:.6f} sec")



Malicious prover times: [0.0012147340003139107, 0.0011638659998425283, 0.001135534000241023, 0.0014322340002763667, 0.001186194999718282, 0.001174449999780336, 0.0012257500002306188, 0.0013164049996703397, 0.0012974520004718215, 0.0014192309999998542]
Average malicious attestation time: 0.001257 sec


In [9]:
def collect_times(prover_function, memory_lines, runs=50):
    times = []
    for _ in range(runs):
        _, att_time, _, _, _ = verifier(prover_function, memory_lines)
        times.append(att_time)
    return times

legit_times = collect_times(prover, random_memory)
malicious_times = collect_times(malicious_wrapper, attack_memory)

print("Legitimate times:", legit_times)
print("Malicious times:", malicious_times)

# Set the threshold as a value between the average legitimate and malicious times.
threshold = (sum(legit_times)/len(legit_times) + sum(malicious_times)/len(malicious_times)) / 2
print(f"Response time threshold: {threshold:.6f} sec")


Legitimate times: [0.001112602999455703, 0.0011190200002602069, 0.0011821629996120464, 0.0018902390002040192, 0.0011743319992092438, 0.001137496000410465, 0.0011754500001188717, 0.0011707399999068002, 0.00112522900053591, 0.0011082730006819475, 0.0011043670001527062, 0.0012275499993847916, 0.0013976829995954176, 0.0012048790003973409, 0.001129045000197948, 0.001231573000040953, 0.0013126070007274393, 0.0011577999994187849, 0.001214130999869667, 0.001124148999224417, 0.001319465000051423, 0.001139279000199167, 0.0011170750003657304, 0.0011249870003666729, 0.0013128139999025734, 0.0011237629996685428, 0.0011109469996881671, 0.0012738900004478637, 0.0011081470001954585, 0.0011769520006055245, 0.0010940630008917651, 0.0010832549996848684, 0.0010776749995784485, 0.0011481790006655501, 0.001136020999183529, 0.0010789069992824807, 0.0010848560004887986, 0.0011118379998151795, 0.0013523690004149103, 0.0011100639994765515, 0.0010932620007224614, 0.0010781419996419572, 0.0011765279996325262, 0.0

In [10]:
def enhanced_verifier(prover_function, memory_lines, time_threshold):
    nonce = generate_nonce()
    start_time = time.perf_counter()
    prover_hash = prover_function(nonce, memory_lines)
    end_time = time.perf_counter()
    attestation_time = end_time - start_time

    expected_hash = prover(nonce, memory_lines)
    passed_hash = (prover_hash == expected_hash)
    # Check timing as an additional defense.
    passed_time = (attestation_time <= time_threshold)
    overall_passed = passed_hash and passed_time
    return overall_passed, attestation_time, passed_hash, passed_time

# Conduct 100 runs (50 of each) to calculate TPR and FPR.
def threshold_detection_analysis(time_threshold):
    legitimate_results = [enhanced_verifier(prover, random_memory, time_threshold)[0] for _ in range(50)]
    malicious_results = [enhanced_verifier(malicious_wrapper, attack_memory, time_threshold)[0] for _ in range(50)]
    
    # True Positive Rate (TPR): legitimate responses accepted.
    TPR = sum(legitimate_results) / len(legitimate_results)
    # False Positive Rate (FPR): malicious responses mistakenly accepted.
    FPR = sum(malicious_results) / len(malicious_results)
    return TPR, FPR

TPR, FPR = threshold_detection_analysis(threshold)
print(f"True Positive Rate: {TPR*100:.2f}%")
print(f"False Positive Rate: {FPR*100:.2f}%")


True Positive Rate: 96.00%
False Positive Rate: 0.00%


In [11]:
def reset_nonces():
    global used_nonces
    used_nonces = set()

def simulate_noise(noise_std):
    """
    Simulate noise as a random delay from a normal distribution with mean 0 and standard deviation noise_std.
    Use the absolute value so the noise delay is always positive.
    """
    noise = abs(random.gauss(0, noise_std))
    return noise

def enhanced_verifier_with_noise(prover_function, memory_lines, time_threshold, noise_std):
    """
    Enhanced verifier that includes noise in the attestation time measurement.
    noise_std: standard deviation of the noise delay (in seconds).
    """
    nonce = generate_nonce()
    start_time = time.perf_counter()
    
    # Introduce simulated noise (network/computation delay)
    noise_delay = simulate_noise(noise_std)
    time.sleep(noise_delay)
    
    # Get the prover's response after the noise delay.
    prover_hash = prover_function(nonce, memory_lines)
    end_time = time.perf_counter()
    
    attestation_time = end_time - start_time
    expected_hash = prover(nonce, memory_lines)
    passed_hash = (prover_hash == expected_hash)
    passed_time = (attestation_time <= time_threshold)
    overall_passed = passed_hash and passed_time
    return overall_passed, attestation_time, passed_hash, passed_time

def noise_simulation_analysis(time_threshold, noise_std_values, runs=100):
    results = []
    for noise_std in noise_std_values:
        reset_nonces()  # Reset nonces for each noise level test
        legitimate_outcomes = [enhanced_verifier_with_noise(prover, random_memory, time_threshold, noise_std)[0]
                               for _ in range(runs)]
        reset_nonces()  # Reset nonces again for the malicious tests
        malicious_outcomes = [enhanced_verifier_with_noise(malicious_wrapper, attack_memory, time_threshold, noise_std)[0]
                              for _ in range(runs)]
        TPR = sum(legitimate_outcomes) / len(legitimate_outcomes)
        FPR = sum(malicious_outcomes) / len(malicious_outcomes)
        results.append((noise_std, TPR, FPR))
        print(f"Noise STD: {noise_std:.6f} sec, TPR: {TPR*100:.2f}%, FPR: {FPR*100:.2f}%")
    return results


# From 0.0005 sec to 0.001 sec with increments of 0.00005 sec.
noise_std_values = np.arange(0.0001, 0.00100, 0.00001).tolist()

print("\n--- Noise Simulation Analysis for Both Legitimate and Malicious Provers ---")
noise_results = noise_simulation_analysis(threshold, noise_std_values, runs=200)

# Find the approximate noise level where TPR falls to ~50%
for noise_std, tpr, fpr in noise_results:
    if abs(tpr - 0.5) < 0.05:  # within 5% of 50%
        print(f"\nTPR is around 50% for noise standard deviation ≈ {noise_std:.6f} sec")
        break




--- Noise Simulation Analysis for Both Legitimate and Malicious Provers ---
Noise STD: 0.000100 sec, TPR: 100.00%, FPR: 0.00%
Noise STD: 0.000110 sec, TPR: 99.50%, FPR: 0.00%
Noise STD: 0.000120 sec, TPR: 100.00%, FPR: 0.00%
Noise STD: 0.000130 sec, TPR: 100.00%, FPR: 0.00%
Noise STD: 0.000140 sec, TPR: 99.00%, FPR: 0.00%
Noise STD: 0.000150 sec, TPR: 99.00%, FPR: 0.00%
Noise STD: 0.000160 sec, TPR: 99.50%, FPR: 0.00%
Noise STD: 0.000170 sec, TPR: 100.00%, FPR: 0.00%
Noise STD: 0.000180 sec, TPR: 98.50%, FPR: 0.00%
Noise STD: 0.000190 sec, TPR: 99.50%, FPR: 0.00%
Noise STD: 0.000200 sec, TPR: 99.00%, FPR: 0.00%
Noise STD: 0.000210 sec, TPR: 97.50%, FPR: 0.00%
Noise STD: 0.000220 sec, TPR: 99.50%, FPR: 0.00%
Noise STD: 0.000230 sec, TPR: 99.00%, FPR: 0.00%
Noise STD: 0.000240 sec, TPR: 98.00%, FPR: 0.00%
Noise STD: 0.000250 sec, TPR: 98.00%, FPR: 0.00%
Noise STD: 0.000260 sec, TPR: 97.50%, FPR: 0.00%
Noise STD: 0.000270 sec, TPR: 95.50%, FPR: 0.00%
Noise STD: 0.000280 sec, TPR: 98.50%,

### Techniques to Improve Attestation Accuracy Despite Noise

- **Statistical Filtering:**  
  - Use multiple measurements of the attestation time and compute statistics (like the mean or median) to reduce the impact of random fluctuations.  
  - Filtering out outliers helps smooth the results and better estimate the true response time.

- **Adaptive Thresholds:**  
  - Instead of a fixed time threshold, adjust the threshold based on observed noise levels and historical performance.  
  - This dynamic approach allows the system to account for varying network and processing conditions, reducing false positives and negatives.

- **Multiple Verification Attempts:**  
  - Perform several attestation checks consecutively rather than relying on a single measurement.  
  - By requiring multiple successful verifications (or averaging the results), transient noise effects can be minimized, leading to a more robust verification process.
